# A3 : Alternating Least Squares

In [2]:
import numpy as np
import pandas as pd
np.random.seed(0)
np.seterr(divide='ignore', invalid='ignore') # supress divide by zero warning

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [3]:
from surprise import Reader, Dataset
from surprise.model_selection import train_test_split
from surprise import BaselineOnly, accuracy
from surprise.accuracy import rmse

In [4]:
filepath = 'C:/Users/lyc07/Desktop/Columbia/S3/GR5243 Applied Data Science/prj4/'
full = pd.read_csv(filepath + 'full.csv')

In [5]:
full.head()

,Unnamed: 0,userId,movieId,rating,timestamp
0,0,1,1,3.0,964982703
1,1,1,3,3.0,964981247
2,2,1,6,3.0,964982224
3,3,1,44,4.0,964983815
4,4,1,47,4.0,964982931


In [6]:
full_data = full[['userId', 'movieId', 'rating']]

In [7]:
full_data.head()

,userId,movieId,rating
0,1,1,3.0
1,1,3,3.0
2,1,6,3.0
3,1,44,4.0
4,1,47,4.0


In [30]:
#train_np = train[['userId','movieId','rating']].values
#train_np = train_np.astype(np.int)
#test_np = test[['userId','movieId','rating']].values
#test_np = test_np.astype(np.int)

In [8]:
num_users = len(full.userId.unique())
num_movies = len(full.movieId.unique())
#num_movies = full['movieId'].unique().max()
print(num_users)
print(num_movies)

610
9724


In [19]:
len(full_data)

100836

In [13]:
reader = Reader()
data = Dataset.load_from_df(full_data, reader)

In [15]:
train, test = train_test_split(data, test_size = 0.2)

In [39]:
min(new_df.rating.unique())

0.5

In [16]:
iterator = train.all_ratings()
new_df = pd.DataFrame(columns=['uid', 'iid', 'rating'])
i = 0
for (uid, iid, rating) in iterator:
    new_df.loc[i] = [uid, iid, rating]
    i = i+1

new_df.head(2)

,uid,iid,rating
0,0.0,0.0,5.0
1,0.0,249.0,4.0


In [42]:
new_df['uid'] = new_df['uid'] + 1
new_df['iid'] = new_df['iid'] + 1

In [43]:
train_df = new_df
train_df.head()

,uid,iid,rating
0,1.0,1.0,5.0
1,1.0,250.0,4.0
2,1.0,375.0,5.0
3,1.0,25.0,5.0
4,1.0,865.0,4.0


In [28]:
test_df = pd.DataFrame(test, columns = ['uid', 'iid', 'rating'])

In [67]:
train_df.to_csv('train_split_by_surprise.csv')
test_df.to_csv('test_split_by_surprise.csv')

In [41]:
min(test_df.rating.unique())

0.0

In [25]:
len(test) + len(new_df)

100836

In [44]:
train_np = train_df[['uid','iid','rating']].values
train_np = train_df.astype(np.int)
test_np = test_df[['uid','iid','rating']].values
test_np = test_df.astype(np.int)

In [20]:
len(new_df)

80668

In [45]:
def format_output(I,M,U):
    approximation = np.multiply(I,np.dot(U.T,M))
    with open('output.txt','w') as f:  
        for i in range(610): # users
            for j in range(9724): # movies
                if approximation[i,j] != 0:
                    f.write("%d\t%d\t%f\n"%(i+1,j+1,approximation[i,j]))

In [46]:
def rmse(I,R,M,U,round=False):
    return np.sqrt(np.sum(np.square(np.multiply(I, (R-np.dot(U.T,M)))))/np.count_nonzero(I))

In [53]:
train_mat = train_np[['uid', 'iid', 'rating']].values
test_mat = test_np[['uid', 'iid', 'rating']].values

In [55]:
train_mat[0]

array([[   1,    1,    5],
       [   1,  250,    4],
       [   1,  375,    5],
       ...,
       [ 610, 1294,    2],
       [ 610,  379,    3],
       [ 610,   68,    3]])

In [56]:
R = np.asmatrix(np.zeros((num_users,num_movies), dtype=np.int)) # sparse rating matrix
for row in train_mat:
    user = row[0] - 1  # users start with id 1
    movie = row[1] - 1
    rating = row[2]
    R[user, movie] = rating

In [57]:
Rtst = np.asmatrix(np.zeros((num_users,num_movies), dtype=np.int))
for row in test_mat:
    user = row[0] - 1
    movie = row[1] - 1
    rating = row[2]
    Rtst[user,movie] = rating

In [58]:
# Indicator Matrices
I = np.copy(R)
I[I > 0] = 1
I[I == 0] = 0
I2 = np.copy(Rtst)
I2[I2 > 0] = 1
I2[I2 == 0] = 0

In [59]:
# ALS Parameters
l = 0.1       # Regularization parameter lambda
k = 0         # convergence iterator
K = 5        # convergence criterion
r = 10      # Dimensionality of latent feature space
m,n = R.shape # Number of users and items

In [60]:
# Initialize matricies
U = 1 * np.random.rand(r,m) # Latent user feature matrix (pattern matrix)
M = 1 * np.random.rand(r,n) # Latent movie feature matrix (coefficient matrix)
avg_movies = np.true_divide(R.sum(0),(R!=0).sum(0))
avg_movies[np.isnan(avg_movies)] = 0
M[0,:] = avg_movies # Set first row of Q to column vector of average ratings
E = np.eye(r,dtype=int) # rxr idendity matrix

In [61]:
train_errors = []
test_errors = []

In [62]:
# Repeat until convergence
while k < K:
    # Fix M and solve for U
    for i, Ii in enumerate(I):
        nui = np.count_nonzero(Ii) # Number of items user i has rated
        if (nui == 0): nui = 1 # remove zeros
    
        # Least squares solution
        Ai = np.add(np.dot(M, np.dot(np.diag(Ii), M.T)), np.multiply(l, np.multiply(nui, E))) # A_i = M_{I_i}M_{I_i}^T + ln_{u_i}E
        Vi = np.dot(M, np.dot(np.diag(Ii), R[i].T)) # V_i = M_{I_i}R^T(i,I_i)
        U[:,i] = np.linalg.solve(Ai,Vi).reshape(r)

    # Fix U and solve for M
    for j, Ij in enumerate(I.T):
        nmj = np.count_nonzero(Ij) # Number of users that rated item j
        if (nmj == 0): nmj = 1 # remove zeros

        # Least squares solution
        Aj = np.add(np.dot(U, np.dot(np.diag(Ij), U.T)), np.multiply(l, np.multiply(nmj, E))) # A_j = U_{I_j}U_{I_j}^T + ln_{m_j}E
        Vj = np.dot(U, np.dot(np.diag(Ij), R[:,j])) # V_j = U_{I_j}R(I_j,j)
        try:
            M[:,j] = np.linalg.solve(Aj,Vj).reshape(r)
        except: 
            print("singular matrix error: " + str(j + 1)) # movie has no reviewers 
            continue

    train_rmse = rmse(I,R,M,U)
    test_rmse = rmse(I2,Rtst,M,U)
    train_errors.append(train_rmse)
    test_errors.append(test_rmse)
    #print "[k: %d/%d] train-RMSE = %f  test-RMSE = %f" %(k+1, K, train_rmse, test_rmse)
    k = k + 1

format_output(I2,M,U)
print(test_errors)

[1.4416814298663398, 1.5084152501456283, 1.5407303593431934, 1.549531615946494, 1.548357351168599]


In [65]:
M
np.save('movie_mat.npy', M)    # .npy extension is added if not given
#d = np.load('movie.npy')

In [66]:
U
np.save('user_mat.npy', U)    # .npy extension is added if not given
#d = np.load('user_mat.npy')